<a href="https://colab.research.google.com/github/apercky/isa/blob/master/IntradaySeasonal-QGC_GOLD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intraday Seasonal Analysis - Gold Future (QGC#C iQfeed)

Inspired by the famous site, this analysis tries to generate a graph that shows the bias on the historical series in a similar way to that famous site.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *

In [0]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
from collections import OrderedDict

## Export data from Multicharts

<img src="https://analysis-cdn.s3.us-east-2.amazonaws.com/Schermata+2019-10-16+alle+15.22.04.png" alt="Drawing" width="600"/>

## Upload you data
Once you exported your data from Multicharts, you havo to upload on Colab Files (*or other path if you run in local environment*)

In [0]:
# Export data from Multicharts at minutes basis
# Upload file in colab Files

# Load from Colab Files
df = pd.read_csv('/content/drive/My Drive/QGC#C-Minute-Trade.txt', 
            delimiter = ',', skiprows=1, 
            names=["Date","Time","Open","High","Low","Close","TotalVolume"])

## Index creation
The Multichart export includes Date and Time as two separate columns. We need to merge them an use the resulting column as index.

In [0]:
# Create the index from Date and Time columns
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format="%m/%d/%Y %H:%M:%S")
df.set_index(df['DateTime'], inplace=True)

# Remove unused columns
del df['Date']
del df['Time']

In [0]:
# Start analysis from 2010
df = df[(df.index.year >= 2010)]

## Data resampling
Resampling data at 15 minutes bars. We also move the start of week to Sunday which is the start of trading week.

In [0]:
# Resampling OHLC 
df_resampled = df.resample('15min').agg({'Open' : 'first', 
                                'High' : 'max', 
                                'Low'  : 'min', 
                                'Close': 'last'})
df_resampled['Time'] = df_resampled.index.time

# Move the start of weekday on Sunday, because normaly it starts on Monday
df_resampled['WeekDay'] = (df_resampled.index.weekday + 1) % 7

df_resampled.dropna(inplace=True)
# df_resampled.plot()

## Mark the starts of the week
We need a marker that makes easy to keep track the first bar of the week. In this section we use "FirstBarOfWeek" column for this goal.

The "Diff" column will be calculate with weekly reset, with value that goes back to zero weekly.

In [0]:
# Create a column with a flag that will be set to 1 at the first bar of week
# It is used for weekly reset in calculation
df_resampled['FirstBarOfWeek'] = 0
df_resampled.loc[(df_resampled['WeekDay'] == 0), 'FirstBarOfWeek'] = 1
df_resampled['FirstBarOfWeek'] = np.where(df_resampled['FirstBarOfWeek'] == df_resampled['FirstBarOfWeek'].shift(1) , 0, 1)

# calculate change that starts from zero any weeks
df_resampled['Diff'] = np.where(df_resampled['FirstBarOfWeek'] ==1 , 0, df_resampled['Close'] - df_resampled['Close'].shift(1))

df_resampled.dropna(inplace=True)


## Cumulative difference calculation 
This will be calculate as weekly reset like the "Diff" column.

In [0]:
# This is a trick, it is useful for reset the cumsum at new week

# Cumsum with reset
df_resampled['reset'] = df_resampled['FirstBarOfWeek'].cumsum()

# Calculate the cumulative sum of differences
df_resampled['DiffCum']= df_resampled.groupby(['reset'])['Diff'].cumsum()

# Remove the reset column
df_resampled = df_resampled.drop(['reset'], axis=1)


## Prepare data for visualization 
Get a copy of dataframe and remove the unused columns. And then, create the aggregation that break data in Week, Week Days and Timing (15min).

In [0]:
# Create a subset of data for printing purpose
df_print = df_resampled.copy()
df_print = df_print.drop(['Open', 'High', 'Low', 'Close', 'FirstBarOfWeek'], axis=1)
df_print.dropna(inplace=True)

# Group you data for week break, week days, and timing of your bar
df_print = df_print.groupby(['WeekDay','Time']).sum()


In [0]:
# Convert in flat by index reset and prepare x labels
weekdays_dic ={0:'Sun', 1:'Mon', 2:'Tue',3:'Wed',4:'Thu',5:'Fri', 6:'Sat'}
df_flat = df_print.reset_index()
df_flat['WeekDayName'] = df_flat['WeekDay'].apply(lambda x: weekdays_dic[x] )
df_flat['x'] = df_flat['WeekDayName'] + ' ' + (df_flat['Time'].astype(str))


## Start with printing

### Print the difference

In [17]:
# Print the difference for aggregated data
fig = go.Figure()

fig.update_xaxes(
    tickmode = 'array',
    ticktext=df_flat['x'],
    tickvals=df_flat.index,
    showticklabels=False,
    tickangle=90, 
    tickfont=dict(family='Rockwell', color='crimson', size=6),
    
)

fig.update_layout(

    title = "Intraday Seasonal - Difference",

)

fig.add_trace(go.Scatter(
    y=df_flat['Diff'],
    marker=dict(color="crimson", size=1),
    mode='lines',
    name="Diff",
))

fig.show()

### Print the cumulative difference (Intraday Seasonal Chart)

In [18]:
# Print the cumulative difference for aggregated data
fig = go.Figure()


fig.update_xaxes(
    tickmode = 'array',
    ticktext=df_flat['x'],
    tickvals=df_flat.index,
    showticklabels=False,
    tickangle=90, 
    tickfont=dict(family='Rockwell', color='crimson', size=6),
    
)

fig.update_layout(

    title = "Intraday Seasonal - Cumulative Difference",

)

fig.add_trace(go.Scatter(
    x=df_flat.index,
    y=df_flat['DiffCum'],
    marker=dict(color="crimson", size=1),
    mode='lines',
    name="DiffCum",
))

fig.show()

## Comparing with Intraday Seasonal

The image bellow was getted from Intraday Seasonals site only for camparing purpose.

![alt text](https://analysis-cdn.s3.us-east-2.amazonaws.com/Schermata+2019-10-16+alle+16.13.14.png)